In [1]:
# Testando JVM
import weka.core.jvm as jvm

# Inicie a JVM sem especificar o caminho para o weka.jar
jvm.start()

DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['c:\\Users\\Luyza\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\weka\\lib\\arpack_combined.jar', 'c:\\Users\\Luyza\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\weka\\lib\\core.jar', 'c:\\Users\\Luyza\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\weka\\lib\\mtj.jar', 'c:\\Users\\Luyza\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'c:\\Users\\Luyza\\AppData\\Local\\Programs\\Python\\Python312\\Lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled


In [2]:
data_dir = "C:\\Users\\Luyza\\Documents\\Dados"

In [3]:
from weka.core.converters import Loader
# Cria uma instância do CSVLoader
loader = Loader(classname="weka.core.converters.CSVLoader")
# Carrega o dataset CSV (substitua pelo nome do seu arquivo CSV)
data = loader.load_file(f"{data_dir}/MSC_calibration.csv")
# Define a última coluna como a classe (se necessário)
data.class_is_last()

In [4]:
from weka.classifiers import Classifier
from weka.classifiers import Evaluation
import numpy as np

# Cria uma instância do classificador RandomForest
# As opções são ajustadas para refletir o que foi especificado no seu exemplo
cls = Classifier(classname="weka.classifiers.trees.RandomForest", options=[
    "-P", "100",  # Percentage of bagging (not used here, but set to 100)
    "-I", "100",  # Number of trees
    "-num-slots", "1",  # Number of threads to use
    "-K", "0",  # Number of attributes to consider (0 means all)
    "-M", "1.0",  # Minimum number of instances per leaf
    "-V", "0.001",  # Percentage of instances to leave out for the random forest model
    "-S", "1"  # Random seed
])

# Treina o classificador com o dataset
cls.build_classifier(data)

# Avalia o modelo com o próprio conjunto de dados de treinamento
evaluation = Evaluation(data)

# Gera previsões para o conjunto de dados de treinamento
predictions = [cls.classify_instance(inst) for inst in data]

# Obtém as classes reais
actual = [inst.get_value(data.class_index) for inst in data]

# Converte para numpy arrays para facilitar o cálculo das métricas
actual = np.array(actual)
predictions = np.array(predictions)

# Calcula as métricas manualmente
mae = np.mean(np.abs(predictions - actual))
rmse = np.sqrt(np.mean((predictions - actual) ** 2))
rae = np.sum(np.abs(predictions - actual)) / np.sum(np.abs(actual - np.mean(actual)))
rrmse = np.sqrt(np.mean((predictions - actual) ** 2)) / np.mean(actual)

# Coeficiente de Correlação
correlation_coefficient = np.corrcoef(predictions, actual)[0, 1]

# Número Total de Instâncias
total_instances = len(data)

# Imprime as métricas de avaliação
print(f"Coeficiente de Correlação: {correlation_coefficient}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"RAE: {rae}")
print(f"RRMSE: {rrmse}")
print(f"Número Total de Instâncias: {total_instances}")

Coeficiente de Correlação: 0.9789283120991824
MAE: 0.0005657088613731934
RMSE: 0.0007377036649620975
RAE: 0.23525020817412567
RRMSE: 0.03445389540483091
Número Total de Instâncias: 175


In [5]:
# Classifica cada instância e imprime os resultados
for index, inst in enumerate(data):
    pred = cls.classify_instance(inst)
    dist = cls.distribution_for_instance(inst)
    #print(f"{index+1}: label index={pred}, class distribution={dist}") -> output das prediçoes
    
# Inicializa a avaliação
evaluation = Evaluation(data)

# Avalia o modelo com o próprio conjunto de dados (cross-validation)
evaluation.evaluate_model(cls, data)

mae = np.mean(np.abs(predictions - actual))
rmse = np.sqrt(np.mean((predictions - actual) ** 2))
rae = np.sum(np.abs(predictions - actual)) / np.sum(np.abs(actual - np.mean(actual)))
rrmse = np.sqrt(np.mean((predictions - actual) ** 2)) / np.mean(actual)

# Coeficiente de Correlação
correlation_coefficient = np.corrcoef(predictions, actual)[0, 1]

# Número Total de Instâncias
total_instances = len(data)

# Imprime as métricas de avaliação
print(f"Coeficiente de Correlação: {correlation_coefficient}")
print(f"MAE: {mae}")
print(f"RMSE: {rmse}")
print(f"RAE: {rae}")
print(f"RRMSE: {rrmse}")
print(f"Número Total de Instâncias: {total_instances}")
print(cls)

TypeError: No matching overloads found for *static* weka.classifiers.Evaluation.evaluateModel(weka.classifiers.trees.RandomForest,Instances), options are:
	public static java.lang.String weka.classifiers.Evaluation.evaluateModel(weka.classifiers.Classifier,java.lang.String[]) throws java.lang.Exception
	public static java.lang.String weka.classifiers.Evaluation.evaluateModel(java.lang.String,java.lang.String[]) throws java.lang.Exception
	public double[] weka.classifiers.Evaluation.evaluateModel(weka.classifiers.Classifier,weka.core.Instances,java.lang.Object[]) throws java.lang.Exception


In [ ]:
import weka.plot.graph as graph  # NB: pygraphviz and PIL are required
graph.plot_dot_graph(cls.graph)